In [5]:
import pandas as pd
import numpy as np 
import os
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, make_scorer, roc_auc_score, mean_squared_error,mean_absolute_error
from sklearn.cluster import KMeans
from scipy import stats
from sklearn.model_selection import GridSearchCV
from prophet.diagnostics import cross_validation
import joblib
import optuna
import catboost
import lightgbm as lgb
from skopt import BayesSearchCV
from prophet import Prophet
from prophet.diagnostics import performance_metrics
import xgboost as xgb

In [6]:
def y_changer(data):
    data = data.rename(columns = {"y": "평균기온", "월":"months"})
    return data

In [7]:
data = pd.read_csv('./prep_data.csv')
data.ds = pd.to_datetime(data.ds)
data = data.rename(columns = {"평균기온":"y"})
data['days'] = data.ds.dt.day
data['years'] = data.ds.dt.year
summit_form = pd.read_csv('./sample_submission.csv')
regressors = [x for x in data.columns if x != "y" and x != "ds"]
data = y_changer(data)

In [8]:
data.columns

Index(['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율',
       '평균기온', 'months', 'ds', 'days', 'years'],
      dtype='object')

In [181]:
X = data.drop(["평균기온",'일교차', '일조합','ds'], axis = 1)
y = data.평균기온

In [33]:
y_new = y[365:]

In [34]:
X_new = X[:-365]

In [12]:
new

22646

In [13]:
len(y_new)

22646

In [40]:
X_train, X_test = X_new[:-365], X_new[-365:]
y_train, y_test = y_new[:-365], y_new[-365:]

In [41]:
X_test

,최고기온,최저기온,강수량,평균습도,평균풍속,일사합,일조율,months,days,years
22281,1.6,-9.8,0.61,64.0,2.0,9.54,67.7,1.0,1,2021
22282,-1.4,-8.4,0.18,38.5,2.6,11.34,93.8,1.0,2,2021
22283,-2.0,-9.1,0.09,45.0,2.0,8.80,56.7,1.0,3,2021
22284,0.3,-8.4,0.00,51.4,1.7,7.44,47.4,1.0,4,2021
22285,-2.1,-9.9,0.00,52.8,2.9,10.46,88.7,1.0,5,2021
...,...,...,...,...,...,...,...,...,...,...
22641,-3.9,-12.9,0.00,60.9,1.7,6.09,39.6,12.0,27,2021
22642,-0.9,-8.5,0.66,73.8,2.2,4.32,17.7,12.0,28,2021
22643,5.9,-3.8,0.20,72.9,2.6,4.64,18.8,12.0,29,2021
22644,0.2,-6.8,0.00,48.5,3.3,9.14,76.0,12.0,30,2021


In [45]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV



# 하이퍼파라미터 탐색 범위 정의
param_space = {
    'learning_rate': (0.01, 1.00, 'uniform'),
    'max_depth': (3, 12),
    'n_estimators': (100, 1000),
    'subsample': (0.10, 1.00, 'uniform'),
    'colsample_bytree': (0.10, 1.00, 'uniform'),
    'gamma': (0, 1.0, 'uniform'),
}

# XGBoost 모델 초기화
model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# BayesSearchCV를 사용하여 베이지안 최적화 수행
opt = BayesSearchCV(
    model,
    param_space,
    scoring='neg_mean_absolute_error',
    n_iter=50,  # 반복 횟수
    cv=5,  # 교차 검증 폴드 수
    n_jobs=1,  # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
)

# 최적의 하이퍼파라미터로 모델 학습
opt.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", opt.best_params_)

# 최적의 모델 평가 (여기서는 교차 검증 점수 출력)
print("Cross-Validation Score:", -opt.best_score_)


Best Parameters: OrderedDict([('colsample_bytree', 0.8780741118210806), ('gamma', 0), ('learning_rate', 0.12473347174963892), ('max_depth', 8), ('n_estimators', 456), ('subsample', 0.6080385860401739)])
Cross-Validation Score: 2.758964698078302


In [47]:
opt.best_estimators

AttributeError: 'BayesSearchCV' object has no attribute 'best_estimators'

In [48]:
best_model = opt.best_estimator_

In [58]:
pred = best_model.predict(X_test)

mean_absolute_error(y_test, pred)

mae = mean_absolute_error(y_test, pred)

In [118]:
def select_label(data, labels, label):

    # X = data.drop(["평균기온",'일교차', '일조합','ds', label], axis = 1)
    X = data[labels].drop([label], axis = 1)
    y = data[label]
    
    return X, y 

def train_test_data(X,y):
    
    X_train, X_test = X[:-365], X[-365:]
    y_train, y_test = y[:-365], y[-365:]
    
    return X_train, X_test, y_train, y_test

def make_model(X_train, y_train):
    
    # 하이퍼파라미터 탐색 범위 정의
    param_space = {
        'learning_rate': (0.01, 1.00, 'uniform'),
        'max_depth': (3, 12),
        'n_estimators': (100, 1000),
        'subsample': (0.10, 1.00, 'uniform'),
        'colsample_bytree': (0.10, 1.00, 'uniform'),
        'gamma': (0, 1.0, 'uniform'),
    }

    # XGBoost 모델 초기화
    model = xgb.XGBRegressor(objective="reg:squarederror", n_jobs = -1, random_state=42)

    # BayesSearchCV를 사용하여 베이지안 최적화 수행
    opt = BayesSearchCV(
        model,
        param_space,
        scoring='neg_mean_absolute_error',
        n_iter=50,  # 반복 횟수
        cv=5,  # 교차 검증 폴드 수
        # n_jobs=1,  # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    )

    # 최적의 하이퍼파라미터로 모델 학습
    opt.fit(X_train, y_train)

    # 최적의 하이퍼파라미터 출력
    print("Best Parameters:", opt.best_params_)

    # 최적의 모델 평가 (여기서는 교차 검증 점수 출력)
    print("Cross-Validation Score:", -opt.best_score_)
    
    return opt
    

def socoring(opt, y_test, X_test):
    
    pred = opt.best_estimator_.predict(X_test)

    mean_absolute_error(y_test, pred)

    mae = mean_absolute_error(y_test, pred)
    print(mae)
    
    return mae

def progress(data, labels):
    
    
    data = data
    
    labels = labels
    
    for label in labels:
    
        X, y = select_label(data, labels, label)

        X_train,X_test, y_train, y_test = train_test_data(X,y)

        opt = make_model(X_train,y_train)

        label = label + "_model"
        
        print(label)

        globals()[label] = opt

        mae = socoring(opt, y_test, X_test)

        return print(mae)

def modeling(params, X_train, y_train):
    
    model = xgb.XGBRegressor(objective="reg:squarederror", **params, n_jobs = -1, random_state=42)
    
    model.fit(X_train, y_train)
    
    return model

def model_predict(data, label, params):
    
    data = data
    
    label = label
    
    X, y = select_label(data, label)
    
    X_train,X_test, y_train, y_test = train_test_data(X,y)
    
    model = modeling(params, X_train, y_train)
    
    model_pred = model.predict(X_test)
    
    return model_pred
    

In [113]:
avg_tmp = dict([('colsample_bytree', 0.7489488665614384), ('gamma', 0), ('learning_rate', 0.2303112080128232), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 1.0)])
high_tmp = dict([('colsample_bytree', 0.785843038567719), ('gamma', 1), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 0.36837214707260824)])
low_tmp = dict([('colsample_bytree', 0.9307064570326005), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 5), ('n_estimators', 1000), ('subsample', 1.0)])
avg_hum = dict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.011022102502535366), ('max_depth', 4), ('n_estimators', 1000), ('subsample', 0.11499728444446672)])

In [114]:
avg_tmp

{'colsample_bytree': 0.7489488665614384,
 'gamma': 0,
 'learning_rate': 0.2303112080128232,
 'max_depth': 3,
 'n_estimators': 1000,
 'subsample': 1.0}

In [102]:
labels = ["평균기온", "최고기온", "최저기온", "평균습도"]
for label in labels :
    progress(data, labels)

Best Parameters: OrderedDict([('colsample_bytree', 0.7489488665614384), ('gamma', 0), ('learning_rate', 0.2303112080128232), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 1.0)])
Cross-Validation Score: 0.39732766430568645
0.3690247294970164
0.3690247294970164
Best Parameters: OrderedDict([('colsample_bytree', 0.785843038567719), ('gamma', 1), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 0.36837214707260824)])
Cross-Validation Score: 0.39090319278662566
0.3763858020795535
0.3763858020795535
Best Parameters: OrderedDict([('colsample_bytree', 0.9307064570326005), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 5), ('n_estimators', 1000), ('subsample', 1.0)])
Cross-Validation Score: 0.3902358750681939
0.37210015173235994
0.37210015173235994
Best Parameters: OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.011022102502535366), ('max_depth', 4), ('n_estimators', 1000), ('subsample', 0.11499728444446672)])
Cross-Valid

In [104]:
print(최고기온_model.best_params_)
print(최저기온_model.best_params_)
print(평균습도_model.best_params_)
print(평균기온_model.best_params_)

OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 1.0)])
OrderedDict([('colsample_bytree', 1.0), ('gamma', 1), ('learning_rate', 0.219519027392961), ('max_depth', 8), ('n_estimators', 1000), ('subsample', 0.5209965334687984)])
OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 5), ('n_estimators', 978), ('subsample', 0.6873473201351862)])
OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.011022102502535366), ('max_depth', 4), ('n_estimators', 1000), ('subsample', 0.11499728444446672)])


In [125]:
label_dic = {"최고기온" : high_tmp, "최저기온" : low_tmp, "평균습도" : avg_hum}

In [126]:

X, y = select_label(data, labels, "평균기온")

X_train,X_test, y_train, y_test = train_test_data(X,y)

base_m = modeling(avg_tmp, X_train, y_train)\

for k,v in label_dic.items() :
    
    X1, y1 = select_label(data, labels, k)
    
    X_train1,X_test1, y_train1, y_test1 = train_test_data(X1,y1)
    
    print(v)
    
    m = modeling(v, X_train1, y_train1)
    
    X_test[k] = m.predict(X_test1)
    

# y_pred = base_m.predict(X_test)

# mae = mean_absolute_error(y_test, pred)

# print(mae)

{'colsample_bytree': 0.785843038567719, 'gamma': 1, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.36837214707260824}
{'colsample_bytree': 0.9307064570326005, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 1000, 'subsample': 1.0}
{'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.011022102502535366, 'max_depth': 4, 'n_estimators': 1000, 'subsample': 0.11499728444446672}


In [127]:
y_pred = base_m.predict(X_test)

mae = mean_absolute_error(y_test, pred)

print(mae)

2.8153716568097678


In [128]:
X_test

,최고기온,최저기온,평균습도
22646,-0.155919,-8.238092,60.601475
22647,3.271398,-5.182496,56.391415
22648,2.114505,-5.983513,57.746731
22649,2.158552,-6.311744,54.248131
22650,1.312312,-6.560971,55.886009
...,...,...,...
23006,1.562991,-6.547873,54.769093
23007,1.144488,-6.849960,56.458641
23008,1.172052,-6.911513,56.207546
23009,2.602852,-5.330601,57.111187


In [ ]:

X, y = select_label(data, labels, "평균기온")

# X_train,X_test, y_train, y_test = train_test_data(X,y)
X_test = pd.DataFrame()

base_m = modeling(avg_tmp, X, y)

for k,v in label_dic.items() :
    
    X1, y1 = select_label(data, labels, k)
    
    # X_train1,X_test1, y_train1, y_test1 = train_test_data(X1,y1)
    
    print(v)
    
    m = modeling(v, X1, y1)
    
    X_test[k] = m.predict(X_test1)
    

# y_pred = base_m.predict(X_test)

# mae = mean_absolute_error(y_test, pred)

# print(mae)

In [139]:
def base_select_label(data, labels, label):
    
    X = data[["years", "months", "days"]]
    y = data[label]
    
    return X, y 

def train_test_data(X,y, level):
    
    X_train, X_test = X[:-level], X[-level:]
    y_train, y_test = y[:-level], y[-level:]
    
    return X_train, X_test, y_train, y_test

def progress(data, labels):
    
    
    data = data
    
    labels = labels
    
    for label in labels:
    
        X, y = select_label(data, labels, label)

        X_train,X_test, y_train, y_test = train_test_data(X,y)

        opt = make_model(X_train,y_train)

        label = label + "_model"
        
        print(label)

        globals()[label] = opt
    
def make_model(X_train, y_train, X_test, y_test):
    
    # 하이퍼파라미터 탐색 범위 정의
    param_space = {
        'learning_rate': (0.0001, 0.1, 'uniform'),
        'max_depth': (3, 12),
        'n_estimators': (500, 2000),
        # 'subsample': (0.10, 1.00, 'uniform'),
        'colsample_bytree': (0.4, 1.00, 'uniform'),
        'gamma': (1, 10.0, 'uniform'),
        'booster': ['dart'],
        'lambda': (1e-10, 0.01, 'uniform'),
        'alpha': (1e-10, 0.01, 'uniform'),
    }

    # XGBoost 모델 초기화
    # model = xgb.XGBRegressor(objective="reg:linear", eval_metric='mae', n_jobs = -1, random_state=42) 베스트 기준
    model = xgb.XGBRegressor(objective="reg:linear", eval_metric='mae', n_jobs = -1, random_state=42)

    # BayesSearchCV를 사용하여 베이지안 최적화 수행
    opt = BayesSearchCV(
        model,
        param_space,
        scoring='neg_mean_absolute_error',
        n_iter=10,  # 반복 횟수
        cv=3,  # 교차 검증 폴드 수
        n_jobs=1,  # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    )
    
    # 최적의 하이퍼파라미터로 모델 학습
    opt.fit(X_train, y_train)

    # 최적의 하이퍼파라미터 출력|
    print("Best Parameters:", opt.best_params_)

    # 최적의 모델 평가 (여기서는 교차 검증 점수 출력)
    print("Cross-Validation Score:", -opt.best_score_)
    
    return opt
    

In [122]:
labels = ["최고기온", "최저기온", "평균습도", "평균풍속","일사합", "일조율", "강수량"]

In [123]:
X['months'] = X['months'].astype(int)

In [37]:
X_train,X_test, y_train, y_test = train_test_data(X,y, 365)

In [83]:
def make_model_opt(X, y, level):
    
    if level > 0 :

        X_train,X_test, y_train, y_test = train_test_data(X,y, level)

        def objective(trial):
            
            params = {
                'objective': 'reg:linear',
                'eval_metric': 'mae',
                'booster': 'dart',
                'lambda': trial.suggest_uniform('lambda', 1e-12, 0.01),
                'alpha': trial.suggest_uniform('alpha', 1e-12, 0.01),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
                'gamma' : trial.suggest_uniform('gamma', 1.0, 5.0),
                'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
                'learning_rate': trial.suggest_uniform('learning_rate', 0.0001, 0.1),
                'n_estimators': trial.suggest_int('n_estimators', 200, 1000)
            }

            model = xgb.XGBRegressor(**params)
            model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=15, verbose=False)
            predictions = model.predict(X_test)
            mae = mean_absolute_error(y_test, predictions)

            return mae
        
    else :
            
        X_train,y_train = X,y 

        def objective(trial):
            params = {
                'objective': 'reg:linear',
                'eval_metric': 'mae',
                'booster': 'dart',
                'lambda': trial.suggest_uniform('lambda', 1e-12, 0.01),
                'alpha': trial.suggest_uniform('alpha', 1e-12, 0.01),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
                'gamma' : trial.suggest_uniform('gamma', 1.0, 5.0),
                'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
                'learning_rate': trial.suggest_uniform('learning_rate', 0.0001, 0.1),
                'n_estimators': trial.suggest_int('n_estimators', 200, 1000)
            }

            model = xgb.XGBRegressor(**params)
            model.fit(X_train, y_train, verbose=False)
            predictions = model.predict(X_test)
            mae = mean_absolute_error(y_test, predictions)

            return mae



    # Optuna 베이지안 최적화를 사용하여 파라미터 튜닝 실행
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
    study.optimize(objective, n_trials=200)

    # 최적의 파라미터 출력
    print('Best trial:')
    print('  Value: {}'.format(study.best_trial.value))
    print('  Params: ')
    for key, value in study.best_trial.params.items():
        print('    {}: {}'.format(key, value))
    
    model = xgb.XGBRegressor(**study.best_trial.params)
    model.fit(X,y,verbose=False)
    
    return model

In [48]:
for label in labels:

    X, y = base_select_label(data, labels, label)
    
    X['months'] = X['months'].astype(int)

    study = make_model_opt(X,y,365)
    
    globals()[label+"_model"] = study

[I 2023-12-27 09:59:37,028] A new study created in memory with name: no-name-5e40fa79-2077-446e-b39d-5849b058968f


[09:59:37] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


[I 2023-12-27 10:00:50,338] Trial 0 finished with value: 10.65012038675073 and parameters: {'lambda': 4.860085171500292e-11, 'alpha': 1.101891276763588e-07, 'max_depth': 9, 'subsample': 0.6135663016476616, 'gamma': 3.865845678878733, 'colsample_bytree': 0.8564401261553833, 'learning_rate': 0.0008264847850506173, 'n_estimators': 756}. Best is trial 0 with value: 10.65012038675073.


[10:00:50] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


[I 2023-12-27 10:02:41,685] Trial 1 finished with value: 4.085873975884425 and parameters: {'lambda': 0.0007733630521394952, 'alpha': 0.004852653181263696, 'max_depth': 12, 'subsample': 0.9931136882423831, 'gamma': 3.241684297031637, 'colsample_bytree': 0.9210506958996316, 'learning_rate': 0.002498389673774015, 'n_estimators': 886}. Best is trial 1 with value: 4.085873975884425.


...

[13:46:08] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[13:46:08] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


[I 2023-12-27 13:46:08,789] Trial 196 finished with value: 5.473755777306752 and parameters: {'lambda': 2.566034911168183e-08, 'alpha': 0.00022566205412985477, 'max_depth': 4, 'subsample': 0.7260720349422414, 'gamma': 1.2150297858670072, 'colsample_bytree': 0.6291322342462218, 'learning_rate': 0.07293636344476831, 'n_estimators': 927}. Best is trial 185 with value: 5.340931887169407.
[I 2023-12-27 13:46:08,944] Trial 197 finished with value: 5.354376383696517 and parameters: {'lambda': 9.514186922965674e-08, 'alpha': 8.280077176080474e-05, 'max_depth': 4, 'subsample': 0.7488875772408868, 'gamma': 1.0713203912222085, 'colsample_bytree': 0.7853346560046582, 'learning_rate': 0.05899148034884224, 'n_estimators': 886}. Best is trial 185 with value: 5.340931887169407.


[13:46:08] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[13:46:09] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


[I 2023-12-27 13:46:09,108] Trial 198 finished with value: 5.346278713801135 and parameters: {'lambda': 3.3424962657004665e-07, 'alpha': 2.1375489554048993e-05, 'max_depth': 4, 'subsample': 0.7632309318295384, 'gamma': 1.4651348530566677, 'colsample_bytree': 0.7568761350360398, 'learning_rate': 0.07674337852278754, 'n_estimators': 917}. Best is trial 185 with value: 5.340931887169407.
[I 2023-12-27 13:46:09,264] Trial 199 finished with value: 5.36120075741206 and parameters: {'lambda': 5.2498675836654204e-08, 'alpha': 3.989461741863749e-05, 'max_depth': 3, 'subsample': 0.7404763999310531, 'gamma': 1.3263141989514027, 'colsample_bytree': 0.8435573976943531, 'learning_rate': 0.09844110670742973, 'n_estimators': 961}. Best is trial 185 with value: 5.340931887169407.


[13:46:09] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
Best trial:
  Value: 5.340931887169407
  Params: 
    lambda: 3.927512140887493e-08
    alpha: 0.00011857638357628308
    max_depth: 4
    subsample: 0.7275057527673366
    gamma: 1.3371993924846948
    colsample_bytree: 0.8033464224579181
    learning_rate: 0.07725506410952598
    n_estimators: 933


In [124]:
for label in labels:

    X, y = base_select_label(data, labels, label)
    
    X['months'] = X['months'].astype(int)

    X_train,X_test, y_train, y_test = train_test_data(X,y, 365)

    opt = make_model(X_train,y_train, X_test, y_test)
    
    globals()[label+"_model"] = opt

[09:04:37] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[09:06:20] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[09:08:07] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[09:09:54] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[09:10:11] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[09:10:28] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:20

In [168]:

import joblib
joblib.dump(최고기온_model, "./models/최고기온_model.joblib")
joblib.dump(최저기온_model, "./models/최저기온_model.joblib")
joblib.dump(평균습도_model, "./models/평균습도_model.joblib")
joblib.dump(평균풍속_model, "./models/평균풍속_model.joblib")
joblib.dump(일사합_model, "./models/일사합_model.joblib")
joblib.dump(일조율_model, "./models/일조율_model.joblib")
joblib.dump(강수량_model, "./models/강수량_model.joblib")
joblib.dump(main_model, f"./models/main_model_{mae}.joblib")


['./models/main_model_2.403804796366917.joblib']

In [171]:
load_model = joblib.load(f"./models/main_model_{mae}.joblib")

In [173]:
최고기온_model.best_params_

OrderedDict([('alpha', 0.0018059954716827217),
             ('booster', 'dart'),
             ('colsample_bytree', 0.9490978031073497),
             ('gamma', 1),
             ('lambda', 0.00566038815935114),
             ('learning_rate', 0.04265225658868159),
             ('max_depth', 3),
             ('n_estimators', 1401)])

In [174]:
최저기온_model.best_params_

OrderedDict([('alpha', 0.002446171382533627),
             ('booster', 'dart'),
             ('colsample_bytree', 0.6259067397333353),
             ('gamma', 2),
             ('lambda', 0.004152401675702935),
             ('learning_rate', 0.07008076550496904),
             ('max_depth', 6),
             ('n_estimators', 1833)])

In [175]:
평균습도_model.best_params_

OrderedDict([('alpha', 0.0012687372890573935),
             ('booster', 'dart'),
             ('colsample_bytree', 0.9947156741012083),
             ('gamma', 4),
             ('lambda', 0.00980586266856425),
             ('learning_rate', 0.0278415294466372),
             ('max_depth', 3),
             ('n_estimators', 503)])

In [176]:
평균풍속_model.best_params_

OrderedDict([('alpha', 0.0022251325703070675),
             ('booster', 'dart'),
             ('colsample_bytree', 0.5292402120830566),
             ('gamma', 5),
             ('lambda', 0.002200259709969314),
             ('learning_rate', 0.0017287263722998417),
             ('max_depth', 5),
             ('n_estimators', 1676)])

In [177]:
일사합_model.best_params_

OrderedDict([('alpha', 0.004520057335882961),
             ('booster', 'dart'),
             ('colsample_bytree', 0.5812703102676536),
             ('gamma', 3),
             ('lambda', 0.007454556964440322),
             ('learning_rate', 0.026209534961167385),
             ('max_depth', 5),
             ('n_estimators', 972)])

In [178]:
일조율_model.best_params_

OrderedDict([('alpha', 0.0019547838527374764),
             ('booster', 'dart'),
             ('colsample_bytree', 0.641752558575229),
             ('gamma', 5),
             ('lambda', 0.004672118281436362),
             ('learning_rate', 0.016095545543492962),
             ('max_depth', 4),
             ('n_estimators', 897)])

In [179]:
강수량_model.best_params_

OrderedDict([('alpha', 0.007462283330969947),
             ('booster', 'dart'),
             ('colsample_bytree', 0.7978947463119102),
             ('gamma', 9),
             ('lambda', 0.0025168651770872096),
             ('learning_rate', 0.0013594270369985855),
             ('max_depth', 5),
             ('n_estimators', 662)])

In [169]:
main_model.best_params_

OrderedDict([('alpha', 0.003355197329838887),
             ('booster', 'dart'),
             ('colsample_bytree', 0.8780136212690809),
             ('gamma', 4),
             ('lambda', 0.006618050992034072),
             ('learning_rate', 0.02676131481250614),
             ('max_depth', 10),
             ('n_estimators', 534)])

In [172]:
load_model.best_params_

OrderedDict([('alpha', 0.003355197329838887),
             ('booster', 'dart'),
             ('colsample_bytree', 0.8780136212690809),
             ('gamma', 4),
             ('lambda', 0.006618050992034072),
             ('learning_rate', 0.02676131481250614),
             ('max_depth', 10),
             ('n_estimators', 534)])

# make_model(X_train, y_train)

In [126]:
from itertools import combinations

def generate_combinations(items):
    all_combinations = []
    for r in range(1, len(items) + 1):
        all_combinations.extend([list(combination) for combination in combinations(items, r)])
    
    return all_combinations

items = ["평균습도", "평균풍속","일사합", "일조율", "강수량"]
result = generate_combinations(items)

In [135]:
minimum = 10

for x in result:
    
    tmp = ["years", "months", "days","최고기온", "최저기온"] + x

    X = data[tmp]
    y = data["평균기온"]
    
    # X_train,X_test, y_train, y_test = train_test_data(X,y, 365)
    X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3) ### 베스트 모델 학습 데이터 
    
    
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=50, verbose=False)
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    
    if mae < minimum :
        
        rs = x
        
        minimum = mae
        
        print(minimum)
        print(rs)

0.39486605627208665
['평균습도']
0.39162381482604663
['평균풍속']
0.3915725303755945
['일사합']
0.3908244495390567
['평균습도', '평균풍속']
0.3840686839883273
['평균습도', '일조율']
0.3804863593780594
['평균풍속', '일조율']
0.37405149063249504
['평균습도', '평균풍속', '일조율']


In [141]:
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일사합", "일조율", "강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일사합","강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속", "일조율", "강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도","일사합","강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속", "강수량"]]
X = data[["years", "months", "days","최고기온", "최저기온","일사합","강수량"]]
y = data["평균기온"]

# X_train,X_test, y_train, y_test = train_test_data(X,y, 365)

In [142]:
X_test['최고기온'] = 최고기온_model.predict(X_test[["years", "months", "days"]])
X_test['최저기온'] = 최저기온_model.predict(X_test[["years", "months", "days"]])
# X_test['평균습도'] = 평균습도_model.predict(X_test[["years", "months", "days"]])
# X_test['평균풍속'] = 평균풍속_model.predict(X_test[["years", "months", "days"]])
X_test['일사합'] = 일사합_model.predict(X_test[["years", "months", "days"]])
# X_test['일조율'] = 일조율_model.predict(X_test[["years", "months", "days"]])
X_test['강수량'] = 강수량_model.predict(X_test[["years", "months", "days"]])

In [27]:
XX_train,XX_test, yy_train, yy_test = train_test_data(X_test,y_test, 365)

In [28]:
len(XX_train)

10585

In [195]:
X_train

,years,months,days,최고기온,최저기온,평균습도,평균풍속,일사합,일조율,강수량
15280,2001,11.0,1,15.9,5.4,55.9,2.7,11.670,67.3,7.00
640,1961,10.0,2,25.1,12.9,57.3,2.3,15.492,64.4,0.04
17099,2006,10.0,25,17.2,9.3,56.6,1.8,7.220,15.6,1.24
1948,1965,5.0,2,13.0,7.5,78.0,1.3,3.457,0.0,0.10
20595,2016,5.0,21,30.4,17.8,23.8,2.0,24.050,86.1,0.62
...,...,...,...,...,...,...,...,...,...,...
6521,1977,11.0,8,11.4,6.2,88.8,2.0,4.280,0.0,24.00
21854,2019,11.0,1,20.5,11.4,57.9,1.8,13.850,88.8,2.14
17608,2008,3.0,17,15.1,2.5,41.5,1.6,15.860,80.0,1.75
16799,2005,12.0,29,1.2,-6.8,48.6,1.5,6.110,65.6,0.62


In [143]:
main_model = make_model(X_train,y_train, X_test, y_test)

[15:21:26] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[15:22:33] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[15:23:49] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[15:25:02] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[15:29:44] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[15:35:07] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:20

In [89]:
params =  {'lambda': 0.007371859201964739, 'alpha': 0.006374389287667976, 'max_depth': 7, 'subsample': 0.667444587973499, 'gamma': 4.271112267726981, 'colsample_bytree': 0.8353925838466474, 'learning_rate': 0.03813381678728731, 'n_estimators': 874}

In [91]:
main_model = xgb.XGBRegressor(**params)
main_model.fit(X,y,verbose=False)

XGBRegressor(alpha=0.006374389287667976, base_score=0.5, booster='gbtree',
             callbacks=None, colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.8353925838466474, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             gamma=4.271112267726981, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             lambda=0.007371859201964739, learning_rate=0.03813381678728731,
             max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=7,
             max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=874, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

##### 일조율 제외 Best Parameters: OrderedDict([('colsample_bytree', 0.8828609870729205), ('gamma', 3), ('learning_rate', 0.10496395941543749), ('max_depth', 7), ('n_estimators', 1760), ('subsample', 0.16884742956302834)]) Cross-Validation Score: 0.38765606817069787

##### 일사합 제외 Best Parameters: OrderedDict([('colsample_bytree', 0.5362786341288028), ('gamma', 2), ('learning_rate', 0.3362751905628303), ('max_depth', 3), ('n_estimators', 1819), ('subsample', 0.5797241067611393)]) Cross-Validation Score: 0.3882969067325051

##### 일사합, 일조율, 강수량 제외 Best Parameters: OrderedDict([('colsample_bytree', 0.8725043517301599), ('gamma', 8), ('learning_rate', 0.05309603877595388), ('max_depth', 12), ('n_estimators', 457), ('subsample', 0.4391305289890164)]) Cross-Validation Score: 0.38738144746374126

##### 일조율, 평균풍속 제외 Best Parameters: OrderedDict([('colsample_bytree', 0.6730910375307264), ('gamma', 1), ('learning_rate', 0.056228955612066124), ('max_depth', 10), ('n_estimators', 868), ('subsample', 0.9840445522115008)]) Cross-Validation Score: 0.39200003326815436

##### 일조, 일사, 풍속 제외 Best Parameters: OrderedDict([('colsample_bytree', 0.17537998974601415), ('gamma', 3), ('learning_rate', 0.3143511882114139), ('max_depth', 4), ('n_estimators', 1980), ('subsample', 0.9483620738655641)])Cross-Validation Score: 0.4345285045763367

In [46]:
mds = [최고기온_model, 최저기온_model, 평균습도_model, 평균풍속_model, 일사합_model,
      일조율_model, 강수량_model]

In [50]:
for m, label in zip(mds, labels):
    
    X, y = base_select_label(data, labels, label)
    
    X['months'] = X['months'].astype(int)

    X_train,X_test, y_train, y_test = train_test_data(X,y, 1095)
    
    model = xgb.XGBRegressor(**m.best_trial.params)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)
    
    globals()[label+"_md"] = model

[16:47:25] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [69]:
X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일조율", "강수량"]]
y = data["평균기온"]

X_train,X_test, y_train, y_test = train_test_data(X,y, 365)

In [70]:
model = xgb.XGBRegressor(**main_model.best_trial.params, eval_metric = "mae")
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)

XGBRegressor(alpha=5.635046283951203e-07, base_score=0.5, booster='dart',
             callbacks=None, colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.9679218656115167, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='mae', gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', lambda=1.620266765011867e-05,
             learning_rate=0.03883120762643073, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=7, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=275, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, ...)

In [71]:
X_test['최고기온'] = 최고기온_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['최저기온'] = 최저기온_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['평균습도'] = 평균습도_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['평균풍속'] = 평균풍속_model.best_estimator_.predict(X_test[["years", "months", "days"]])
# X_test['일사합'] = 일사합_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['일조율'] = 일조율_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['강수량'] = 강수량_model.best_estimator_.predict(X_test[["years", "months", "days"]])

AttributeError: 'Study' object has no attribute 'best_estimator_'

In [158]:
y_pred = main_model.predict(XX_test)

mae = mean_absolute_error(y_test, y_pred)

print(mae)

## 7개 : 2.6142990210754413
## 일조율 제외 : 2.517694694352885
## 일사합 제외 : 2.504838254349179
## 일사합, 일조율, 강수량 제외 : 2.4861543731534317
## 일조율, 평균풍속 제외 : 2.515234800046437
## 일사합, 일조율, 풍속 제외 : 2.4652675723866238
## 강수량 제외 : 2.5520137255893993
## 일사/일조 제외 : 2.5530409243228296

ValueError: Feature shape mismatch, expected: 10, got 9

In [159]:
y_pred = main_model.best_estimator_.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

print(mae)

2.403804796366917


In [155]:
X_test

,years,months,days,최고기온,최저기온,평균습도,평균풍속,일사합,일조율,강수량
4303,1971,10.0,13,20.413063,9.437632,55.3,0.6,10.087179,87.7,2.575866
22404,2021,5.0,4,20.651192,14.105269,82.6,2.1,19.678989,0.0,2.595095
1431,1963,12.0,2,6.828722,-4.927246,70.0,1.5,5.307855,50.0,1.691279
15236,2001,9.0,18,25.734510,17.582197,52.8,1.9,12.209292,55.6,3.889452
17658,2008,5.0,6,22.538212,13.819343,30.1,4.0,17.794950,73.4,2.827613
...,...,...,...,...,...,...,...,...,...,...
13062,1995,10.0,6,21.771324,10.059071,40.3,1.9,10.024022,88.8,2.268097
3790,1970,5.0,18,22.687559,12.033097,53.5,2.2,16.264578,76.8,3.060116
13054,1995,9.0,28,23.325029,17.020336,68.3,1.6,11.623250,71.7,3.506213
13929,1998,2.0,19,7.706956,-2.531275,43.3,2.7,8.569327,10.0,1.378729


In [152]:
forms = summit_form.copy()
forms.일시 = pd.to_datetime(forms.일시)
forms['days'] = forms.일시.dt.day
forms['months'] = forms.일시.dt.month
forms['years']= forms.일시.dt.year
forms.drop(['일시','평균기온'], axis = 1, inplace = True)

In [156]:
forms['최고기온'] = 최고기온_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['최저기온'] = 최저기온_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['평균습도'] = 평균습도_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['평균풍속'] = 평균풍속_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['일사합'] = 일사합_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['일조율'] = 일조율_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['강수량'] = 강수량_model.best_estimator_.predict(forms[["years", "months", "days"]])

In [148]:
forms['최고기온'] = 최고기온_model.predict(forms[["years", "months", "days"]])
forms['최저기온'] = 최저기온_model.predict(forms[["years", "months", "days"]])
forms['평균습도'] = 평균습도_model.predict(forms[["years", "months", "days"]])
forms['평균풍속'] = 평균풍속_model.predict(forms[["years", "months", "days"]])
# forms['일사합'] = 일사합_md.predict(forms[["years", "months", "days"]])
# forms['일조율'] = 일조율_model.predict(forms[["years", "months", "days"]])
forms['강수량'] = 강수량_model.predict(forms[["years", "months", "days"]])

In [67]:
import joblib
# joblib.dump(최고기온_model, "./models/최고기온_1.joblib")
# joblib.dump(최저기온_model, "./models/최저기온_1.joblib")
# joblib.dump(평균습도_model, "./models/평균습도_1.joblib")
joblib.dump(main_model, "./models/mainmodel(풍속_일조_일사_제외)_1.joblib")

['./models/mainmodel(풍속_일조_일사_제외)_1.joblib']

In [189]:
# answer = main_model.predict(forms)
answer = trial_model.predict(forms)
summit_form["평균기온"] = answer


# mae = mean_absolute_error(y_test, y_pred)

# print(mae)

In [190]:
summit_form.to_csv(f"./answer/xgb_10vars_max.csv", index = False)

In [4]:
bm = joblib.load("./models/mainmodel(일사합제외)_1.joblib")

In [6]:
bm.best_params_

OrderedDict([('colsample_bytree', 0.5362786341288028),
             ('gamma', 2),
             ('learning_rate', 0.3362751905628303),
             ('max_depth', 3),
             ('n_estimators', 1819),
             ('subsample', 0.5797241067611393)])

In [82]:
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일사합", "일조율", "강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일사합","강수량"]]
X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속", "일조율", "강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도","일사합","강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균풍속"]]
# X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "일조율", "강수량"]]
# X = data[["years", "months", "days","최고기온", "최저기온","평균풍속","일조율", "강수량"]]
y = data["평균기온"]

X_train,X_test, y_train, y_test = train_test_data(X,y, 365)

In [83]:
model = xgb.XGBRegressor(objective="reg:squarederror", **main_model.best_params_, n_jobs = -1, random_state=42)
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.5362786341288028, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=2, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.3362751905628303,
             max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=3,
             max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1819, n_jobs=-1,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [84]:
X_test['최고기온'] = 최고기온_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['최저기온'] = 최저기온_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['평균습도'] = 평균습도_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['평균풍속'] = 평균풍속_model.best_estimator_.predict(X_test[["years", "months", "days"]])
# X_test['일사합'] = 일사합_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['일조율'] = 일조율_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['강수량'] = 강수량_model.best_estimator_.predict(X_test[["years", "months", "days"]])

In [77]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

print(mae)

2.5299242353878197


In [12]:
# 시퀀스 데이터 생성
sequence_length = 180
sequences = []
for i in range(len(X) - sequence_length):
    sequences.append(X.iloc[i:i+sequence_length+1])

In [14]:
sequences[0]

,최고기온,최저기온,강수량,평균습도,평균풍속,일사합,일조율,months,days,years
0,2.2,-5.2,0.34,68.3,1.7,7.029,63.10,1.0,1,1960
1,1.2,-5.6,0.40,87.7,1.3,2.887,4.19,1.0,2,1960
2,8.7,-2.1,0.00,81.3,3.0,5.854,12.87,1.0,3,1960
3,10.8,1.2,0.00,79.7,4.4,6.279,21.55,1.0,4,1960
4,1.3,-8.2,0.05,44.0,5.1,13.086,81.51,1.0,5,1960
...,...,...,...,...,...,...,...,...,...,...
176,31.2,17.8,4.24,61.0,0.8,18.546,54.65,6.0,25,1960
177,25.2,18.8,47.90,89.0,4.8,4.810,2.04,6.0,26,1960
178,24.4,18.0,34.80,89.7,1.5,4.099,1.70,6.0,27,1960
179,24.0,19.6,135.30,98.3,5.9,1.625,0.62,6.0,28,1960


In [183]:
X['months'] = X['months'].astype(int)

In [185]:
X.columns

Index(['최고기온', '최저기온', '강수량', '평균습도', '평균풍속', '일사합', '일조율', 'months', 'days',
       'years'],
      dtype='object')

In [186]:
X_test.columns

Index(['years', 'months', 'days', '최고기온', '최저기온', '평균습도', '평균풍속', '일사합', '일조율',
       '강수량'],
      dtype='object')

In [187]:
trial_model = xgb.XGBRegressor(objective="reg:linear", eval_metric='mae',**main_model.best_params_, n_jobs = -1, random_state=42)
trial_model.fit(X, y)

[16:50:53] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=0.003355197329838887, base_score=0.5, booster='dart',
             callbacks=None, colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.8780136212690809, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='mae', gamma=4, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', lambda=0.006618050992034072,
             learning_rate=0.02676131481250614, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=10, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=534, n_jobs=-1, num_parallel_tree=1,
             objective='reg:linear', predictor='auto', ...)

In [192]:
main_model.best_estimator_

XGBRegressor(alpha=0.003355197329838887, base_score=0.5, booster='dart',
             callbacks=None, colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.8780136212690809, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='mae', gamma=4, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', lambda=0.006618050992034072,
             learning_rate=0.02676131481250614, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=10, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=534, n_jobs=-1, num_parallel_tree=1,
             objective='reg:linear', predictor='auto', ...)

1. 변수 10개 사용
2. 7:3으로 분리, 시드 없음
3. 최고/최저/일사/강수만 예측값으로 test 값을 대체함
4. 검증셋 설정, seed = 42 학습, metrics = mae